In [1]:
from load_data import loading_and_pre_processing_pipeline

feature_data = loading_and_pre_processing_pipeline()
cleaned_feature_data = feature_data.dropna(axis=0)

print(f'{len(feature_data)} records from {len(set(cleaned_feature_data["user_id"].values))} users present. ')
print(f'{len(cleaned_feature_data)} of which are complete.')

cached


/home/kolb/code/rki/ml-nowcasting/Detection model (logistic regression)/load_data.py:161: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  variant_data = variant_data.append({'test_week_start': pd.to_datetime(datetime.now().date()), delta_column: 0, omicronba1_column: 1, omicronba2_column: 99}, ignore_index=True)


293550 records from 7276 users present. 
63516 of which are complete.


In [2]:
feature_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293550 entries, 0 to 293549
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   test_week_start          293550 non-null  datetime64[ns]
 1   omicronba1_share         290159 non-null  float64       
 2   omicronba2_share         290159 non-null  float64       
 3   delta_share              290159 non-null  float64       
 4   user_id                  293310 non-null  float64       
 5   rhr_metric               253716 non-null  float64       
 6   steps_metric             263105 non-null  float64       
 7   sleep_duration_metric    165160 non-null  float64       
 8   vaccination_status       224961 non-null  float64       
 9   days_since_last_dose     220038 non-null  float64       
 10  chills                   286384 non-null  object        
 11  body_pain                286384 non-null  object        
 12  loss_of_taste_an

In [3]:
df = feature_data.dropna(axis=0, subset=['test_result']).drop(columns=['test_result', 'user_id', 'test_week_start', 'date'])
y = feature_data.dropna(axis=0, subset=['test_result'])[['test_result']].astype(bool)

classes_count = y.value_counts()
classes_count

test_result
False          155371
True             7319
dtype: int64

In [4]:
df

,omicronba1_share,omicronba2_share,delta_share,rhr_metric,steps_metric,sleep_duration_metric,vaccination_status,days_since_last_dose,chills,body_pain,loss_of_taste_and_smell,fatigue,cough,cold,diarrhea,sore_throat,asymptomatic,fittness,sex,age
0,0.0,0.0,0.0,0.812354,-0.592758,NaN,0.0,NaN,None,None,None,None,None,None,None,None,None,380.0,773.0,814.0
2,0.0,0.0,0.0,1.362409,NaN,NaN,0.0,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN
3,0.0,0.0,0.0,-0.329008,-0.220081,0.531772,0.0,NaN,None,None,None,None,None,None,None,None,None,380.0,773.0,816.0
4,0.0,0.0,0.0,0.269898,-0.871581,NaN,0.0,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN
5,0.0,0.0,0.0,0.851576,-0.088174,NaN,0.0,NaN,None,None,None,None,None,None,None,None,None,379.0,774.0,816.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293545,NaN,NaN,NaN,1.131692,0.275162,NaN,0.0,NaN,None,None,None,None,None,None,None,None,None,379.0,774.0,817.0
293546,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,None,None,None,None,None,None,None,None,None,496.0,774.0,816.0
293547,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,None,None,None,None,None,None,None,None,None,496.0,774.0,816.0
293548,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,None,None,None,None,None,None,None,None,None,380.0,774.0,813.0


As expected, the dataset is heavily imbalanced with a much higher number of negative than positive test results.
The problems with this are illustrated below:

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyClassifier
import pandas as pd

dummy_clf = DummyClassifier(strategy="most_frequent")
scoring = ["accuracy", "balanced_accuracy"]

index = []
scores = {"Accuracy": [], "Balanced accuracy": []}

# Score a dummy classifier as baseline
index += ["Dummy classifier"]
cv_result = cross_validate(dummy_clf, df, y, scoring=scoring)
scores["Accuracy"].append(cv_result["test_accuracy"].mean())
scores["Balanced accuracy"].append(cv_result["test_balanced_accuracy"].mean())
print(f"Accuracy score of a dummy classifier: {cv_result['test_accuracy'].mean():.3f}")

pd.DataFrame(scores, index=index)

Accuracy score of a dummy classifier: 0.955


,Accuracy,Balanced accuracy
Dummy classifier,0.955013,0.5


This shows that a dummy classifier that classifies all records as the majority class has very high accuracy, simply because it classifies most of the cases correctly. If one corrects for the class imbalance, the accuracy is still 0.5 (random).

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import LogisticRegression

num_pipe = make_pipeline(
    StandardScaler(), SimpleImputer(strategy="mean", add_indicator=True)
)
cat_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value=False),
    OrdinalEncoder(),
)
preprocessor_linear = make_column_transformer(
    [num_pipe, selector(dtype_include="number")],
    [cat_pipe, selector(dtype_include="object")],
    n_jobs=2,
)
lr_clf = make_pipeline(preprocessor_linear, LogisticRegression(max_iter=1000))

In [7]:
# Train and score logistic regression
index += ["Logistic regression"]
cv_result = cross_validate(lr_clf, df, y, scoring=scoring)
scores["Accuracy"].append(cv_result["test_accuracy"].mean())
scores["Balanced accuracy"].append(cv_result["test_balanced_accuracy"].mean())

pd.DataFrame(scores, index=index)

/home/kolb/.cache/pypoetry/virtualenvs/ml-nowcasting-jKKypzXv-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kolb/.cache/pypoetry/virtualenvs/ml-nowcasting-jKKypzXv-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kolb/.cache/pypoetry/virtualenvs/ml-nowcasting-jKKypzXv-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kolb/.cache/pypoetry/virtualenvs/ml-nowc

,Accuracy,Balanced accuracy
Dummy classifier,0.955013,0.500000
Logistic regression,0.958854,0.679055


So the accuracy of the logistic regression is essentially the same as a dummy baseline, if it is not corrected for the imbalance of classes in the dataset. Correcting for imbalance, the accuracy is better than chance, but still not very high. There are two things to improve the performance of the model:
* First, subsampling the majority class during training to get a balanced training set and
* Second, adjusting the loss function of the classifier to weigh cases of the minority class higher.

In [8]:
lr_clf.set_params(logisticregression__class_weight="balanced")

index += ["Logistic regression with balanced class weights"]
cv_result = cross_validate(lr_clf, df, y, scoring=scoring)
scores["Accuracy"].append(cv_result["test_accuracy"].mean())
scores["Balanced accuracy"].append(cv_result["test_balanced_accuracy"].mean())

pd.DataFrame(scores, index=index)

/home/kolb/.cache/pypoetry/virtualenvs/ml-nowcasting-jKKypzXv-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kolb/.cache/pypoetry/virtualenvs/ml-nowcasting-jKKypzXv-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kolb/.cache/pypoetry/virtualenvs/ml-nowcasting-jKKypzXv-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kolb/.cache/pypoetry/virtualenvs/ml-nowc

,Accuracy,Balanced accuracy
Dummy classifier,0.955013,0.500000
Logistic regression,0.958854,0.679055
Logistic regression with balanced class weights,0.789993,0.786117


In [9]:
from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler
from imblearn.under_sampling import RandomUnderSampler

lr_clf = make_pipeline_with_sampler(
    preprocessor_linear,
    RandomUnderSampler(random_state=42),
    LogisticRegression(max_iter=1000),
)

In [10]:
index += ["Under-sampling + Logistic regression"]
cv_result = cross_validate(lr_clf, df, y, scoring=scoring)
scores["Accuracy"].append(cv_result["test_accuracy"].mean())
scores["Balanced accuracy"].append(cv_result["test_balanced_accuracy"].mean())

pd.DataFrame(scores, index=index)

/home/kolb/.cache/pypoetry/virtualenvs/ml-nowcasting-jKKypzXv-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kolb/.cache/pypoetry/virtualenvs/ml-nowcasting-jKKypzXv-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kolb/.cache/pypoetry/virtualenvs/ml-nowcasting-jKKypzXv-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kolb/.cache/pypoetry/virtualenvs/ml-nowc

,Accuracy,Balanced accuracy
Dummy classifier,0.955013,0.500000
Logistic regression,0.958854,0.679055
Logistic regression with balanced class weights,0.789993,0.786117
Under-sampling + Logistic regression,0.789919,0.796565


The performance of both options is comparable and substantially better than the naive regression approach in terms of balanced accuracy.